In [ ]:
!pip install transformers

# Setup Colab GPUs and Allowing Torch to Detect It

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

#The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    # raise SystemError('GPU device not found')
    pass

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


# Downlod your dataset below and parse it


In [ ]:
import pandas as pd
df = pd.read_csv("LEE_scraped_data.csv",engine="python", encoding='utf-8', error_bad_lines=False)

In [ ]:
import numpy as np
df = df.replace(np.nan, "", regex=True)
len(df)

5004

In [ ]:
title = df["title"].tolist()
content = df["content"].tolist()
# comment = df["comment"].tolist()
category = df["category"].tolist()
text_input = []
category_input = []

category_dict =['T Replacement', 'Supplements and Nutrition', 'Training Log', 'Powerful Women', 'Combat', 'Injuries and Rehab']

for i in range(len(df)):
    # category encoding as category_input
    text_input.append(title[i] + " " + content[i]) # + " " + comment[i])
    category_input.append(category_dict.index(category[i]))

print (len(text_input))
print (len(category_input))

5004
5004


In [ ]:
# entries in the dataset
#data=list(df)

In [ ]:
#len(data)

# Tokenization using BERT Tokenizer

In [ ]:
import transformers as ppb
from transformers import BertTokenizer
import torch

In [ ]:
!python3 --version

Python 3.6.9


In [ ]:
# MODELS = [(BertModel),BertTokenizer,'bert-base-uncased']

model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Apply the Tokenizer to One Sentence And Check the Output

In [ ]:
# Print the original sentence.
print(' Original: ', text_input[0] )

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(text_input[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_input[0])))

 Original:  TRT Credentials Thread This was an idea presented by @Chris_Colucci
Here is an opportunity to provide some form of credibility to others.  You can answer as many of these questions as you’d like.  Copy and paste these into your own comment which will become your own entry (you can edit it when required).
THIS IS FOR CREDENTIALS ONLY AND NOT FOR DISCUSSION PLEASE
Go through the list of members and ‘like’ the ones who you feel are providing relevant, factual information.
Name:
Age:
Height:
Weight:
Photo:
Formal education:
Medical professional?:
Facebook:
YouTube:
Instagram:
Other social media:
Years on TRT:
Years training:
Current protocol:
Total and free T:
Tell us something about yourself and your knowledge level:
Tokenized:  ['tr', '##t', 'credentials', 'thread', 'this', 'was', 'an', 'idea', 'presented', 'by', '@', 'chris', '_', 'col', '##ucci', 'here', 'is', 'an', 'opportunity', 'to', 'provide', 'some', 'form', 'of', 'credibility', 'to', 'others', '.', 'you', 'can', 'answ

# Special Tokens
**`[SEP]`**

At the end of every sentence, we need to append the special `[SEP]` token. 

This token is an artifact of two-sentence tasks, where BERT is given two separate sentences and asked to determine something (e.g., can the answer to the question in sentence A be found in sentence B?). 

**`[CLS]`**

For classification tasks, we must prepend the special `[CLS]` token to the beginning of every sentence.

This token has special significance. BERT consists of 12 Transformer layers. Each transformer takes in a list of token embeddings, and produces the same number of embeddings on the output (but with the feature values changed, of course!).

![Illustration of CLS token purpose](http://www.mccormickml.com/assets/BERT/CLS_token_500x606.png)

# Now Tokenize the Entire Dataset

In [ ]:
max_len = 0
# Tokenize all of the sentences and map the tokens to their word IDs.
total_encoded = [] # list of encoded sentences

# Create a for loop in which you 
  # tokenize the text and also add `[CLS]` and `[SEP]` tokens
  # update the maximum sentence length
for i in text_input:
  encoded_sent = tokenizer.encode(i,add_special_tokens=True) # adding special tokens (cls & sep)
  # Add the encoded sentence to the list.
  total_encoded.append(encoded_sent)

# finding max length   
max_len=max([len(i) for i in total_encoded])

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (641 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (765 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (959 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (937 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (971 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Max sentence length:  5332


Actual tokenization now:

The `tokenizer.encode_plus` function combines multiple steps for us:

1. Split the sentence into tokens.
2. Add the special `[CLS]` and `[SEP]` tokens.
3. Map the tokens to their IDs.
4. Pad or truncate all sentences to the same length.
5. Create the attention masks which explicitly differentiate real tokens from `[PAD]` tokens.

The first four features are in `tokenizer.encode`, but use `tokenizer.encode_plus` to get the fifth item (attention masks). Documentation is [here](https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus).

In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

# Create a for loop to use the tokenizer.encode_plus() method on an encoded_dict and 
# set parameters for `sent`, `add_special_tokens`, `max_length`, `pad_to_max_length`, `return_attention_mask`, and `return_tensors`
for text in text_input:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    # the parameters (listed above) should:
                         # Sentence to encode.
                         # Add '[CLS]' and '[SEP]'
                         # Pad & truncate all sentences.                
                         # Construct attn. masks.
                        # Return pytorch tensors.
                 
    
    # Add the encoded sentence to the list.    
    # Use input_ids.append()
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    # USe attention_masks.append()
    attention_masks.append(encoded_dict['attention_mask'])
    
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
category_input = torch.tensor(category_input)

# Print sentence 0, now as a list of IDs.
print('Original: ', text_input[0])
print('Token IDs:', input_ids[0])

Original:  TRT Credentials Thread This was an idea presented by @Chris_Colucci
Here is an opportunity to provide some form of credibility to others.  You can answer as many of these questions as you’d like.  Copy and paste these into your own comment which will become your own entry (you can edit it when required).
THIS IS FOR CREDENTIALS ONLY AND NOT FOR DISCUSSION PLEASE
Go through the list of members and ‘like’ the ones who you feel are providing relevant, factual information.
Name:
Age:
Height:
Weight:
Photo:
Formal education:
Medical professional?:
Facebook:
YouTube:
Instagram:
Other social media:
Years on TRT:
Years training:
Current protocol:
Total and free T:
Tell us something about yourself and your knowledge level:
Token IDs: tensor([  101, 19817,  2102, 22496, 11689,  2023,  2001,  2019,  2801,  3591,
         2011,  1030,  3782,  1035,  8902, 16835,  2182,  2003,  2019,  4495,
         2000,  3073,  2070,  2433,  1997, 21553,  2000,  2500,  1012,  2017,
         2064,  3437

# Splitting Training and Testing Data

90% for training and 10% for classification

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, category_input)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,503 training samples
  501 validation samples


Creating an iterator for our dataset using the torch DataLoader class, which will help save memory while training

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

# Training our Forum Classification Model

We'll be using the [BertForSequenceClassification](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#bertforsequenceclassification) class for fine tuning via the HuggingFace Pytorch implementation. This is the normal BERT model with an added single linear layer on top for classification that we will use as a sentence classifier. As we feed input data, the entire pre-trained BERT model and the additional untrained classification layer is trained on our specific task.

the fine-tuning classes include:
* BertModel
* BertForPreTraining
* BertForMaskedLM
* BertForNextSentencePrediction
* BertForSequenceClassification
* BertForTokenClassification
* BertForQuestionAnswering

[Documentation](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html)

We'll be loading the BERT model using "bert-base-uncased", which only uses lowercased words and phrases

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Below, load BertForSequenceClassification class, the pretrained BERT model with a single linear classification layer on top
# Use the 12-layer BERT model, with an uncased vocab, with two output labels for binary classification
# Set boolean values for output_attentions and output_hidden states

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels = 6,     # must be adjusted to number of categories/labels              
                                                      output_attentions = False, 
                                                      output_hidden_states = False, 
                                                     )


# Tell pytorch to run this model on the GPU.
model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Now we want to get the model's parameters by printing out names and dimensions of the weights for:
1. The embedding layer.
2. The first of the twelve transformers.
3. The output layer.

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

# Use a loop here to print parameters within the list of tuples


print('\n==== First Transformer ====\n')

# Use a loop here to print parameters


print('\n==== Output Layer ====\n')

# Use a loop here to print parameters within the list of tuples


The BERT model has 201 different named parameters.

==== Embedding Layer ====


==== First Transformer ====


==== Output Layer ====



# Adding the Optimizer & Learning Rate Scheduler


We need to grab the training hyperparameters from within the stored model

For the purposes of fine-tuning, the authors recommend choosing from the following values (from Appendix A.3 of the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf)):

>- **Batch size:** 16, 32  
- **Learning rate (Adam):** 5e-5, 3e-5, 2e-5  
- **Number of epochs:** 2, 3, 4 

We chose:
* Batch size: 32 (set when creating our DataLoaders)
* Learning rate: 2e-5
* Epochs: 4 (we'll see that this is probably too many...)

The epsilon parameter `eps = 1e-8` is "a very small number to prevent any division by zero in the implementation" (from [here](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)).

In [ ]:
# # Use the AdamW class from the huggingface library to create an optimizer

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Start with 4 training epochs, but this will likely needed to be changed later
epochs = 4

# Total number of training steps is [number of batches] (use training data loader) x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Build a Training Loop
So what's going on in the training loop?
**Training Phase:**
- Unpack our data inputs and labels
- Load data onto the GPU for acceleration
- Clear out the gradients calculated in the previous pass. 
    - In pytorch the gradients accumulate by default (useful for things like RNNs) unless you explicitly clear them out.
- Forward pass (feed input data through the network)
- Backward pass (backpropagation)
- Tell the network to update parameters with optimizer.step()
- Track variables for monitoring progress

**Evalution Phase:**
- Unpack our data inputs and labels
- Load data onto the GPU for acceleration
- Forward pass (feed input data through the network)
- Compute loss on our validation data and track variables for monitoring progress

Helper fuction to track accuracy

In [ ]:
import numpy as np

# Calculates the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Helper function to format elapsed time

In [ ]:
# Seconds -> hh:mm:ss
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Main Training Loop Below

In [ ]:
import random
import numpy as np

CUDA_LAUNCH_BLOCKING=1

# Set the seed value at 42
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Array to store parameters such as training and validation loss, validation accuracy, and timings
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Backward pass
        model.zero_grad()        

        # Forward pass
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches to calculate average loss
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Use optimizer to update parameters
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # get logits values
            # logits are the output values prior to applying an activation function like the softmax
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...


Training Stats of Each Epoch

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

# Testing the model with our testing data

In [ ]:
# based on https://gist.github.com/michelkana/9252d6d8645b01f1bcce5040c7fd87cd

## Prediction on test set
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()  
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
  
number_correct_classify = 0
  
for i in range(len(predictions)):
  for j in range(len(predictions[i])):
    prediction = np.argmax(predictions[i][j])
    if prediction == true_labels[i][j]:
      number_correct_classify = number_correct_classify + 1

print ('posts_classified_correctly: ',  posts_classified_correctly)

print ('Accuracy on test dataset: {0:.2f}'.format(number_correct_classify/len(input_ids_test)))
## Import and evaluate each test batch using Matthew's correlation coefficient
# from sklearn.metrics import matthews_corrcoef
# matthews_set = []
# for i in range(len(true_labels)):
#   matthews = matthews_corrcoef(true_labels[i],
#                  np.argmax(predictions[i], axis=1).flatten())
#   matthews_set.append(matthews)
  
# # Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
# flat_predictions = [item for sublist in predictions for item in sublist]
# flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# flat_true_labels = [item for sublist in true_labels for item in sublist]

# print('Classification accuracy using BERT Fine Tuning: {0:0.2%}'.format(matthews_corrcoef(flat_true_labels, flat_predictions)))